In [71]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [72]:
# EXTRACT DATA

publisher=pd.read_csv("steam_games.csv")

Hours_data=pd.read_csv("steam-200k.csv", header=None)

Game_features =pd.read_csv("games-features.csv")


In [73]:
# extracting data for publishers
# TRANSFORM DATASETS
Publisher_data = publisher[['publisher','name', 'developer', 'popular_tags', 'minimum_requirements', 'original_price']]
Publisher_data.dropna(how='any',inplace=True)
#remaming columns
Publisher_data = Publisher_data.rename(columns={'name':'gamename', 'developer':'developers','publisher':'publisher', 'popular_tags':'popular_tags','minimum_requirements':'minreqs','original_price':'price'})
Publisher_data = Publisher_data[['gamename', 'developers', 'publisher', 'popular_tags', 'minreqs', 'price']]


/Users/z/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [74]:
# TRANSFORM DATASETS
# Getting data for the number of hours each games are played
Game_number_hour = Hours_data.rename(columns={0:'userid',1:'gamename', 2:'subscriptiontype', 3:'hours'})
Game_number_hour = Game_number_hour[['userid','gamename','subscriptiontype','hours']]
Game_number_hour = Game_number_hour.loc[Game_number_hour['subscriptiontype']== 'play']
Game_number_hour = Game_number_hour.groupby('gamename').count()['hours'].sort_values(ascending=False)
Game_number_hour = pd.DataFrame(Game_number_hour)
Game_number_hour = Game_number_hour.reset_index()
Game_number_hour = Game_number_hour.reset_index()
Game_hours_df =Game_number_hour.rename(columns={"index": "gameid", "gamename": "gamename", "hours": "gamehours"})
Game_hours_df.head(5)

,gameid,gamename,gamehours
0,0,Dota 2,4841
1,1,Team Fortress 2,2323
2,2,Counter-Strike Global Offensive,1377
3,3,Unturned,1069
4,4,Left 4 Dead 2,801


In [75]:
# TRANSFORM DATASETS
# getting features , game requirement and recommendations
Game_feature = Game_features[['ResponseName','Metacritic','RecommendationCount','IsFree','PCReqsHaveMin','PriceInitial']]
Game_feature_df = Game_feature.rename(columns={'ResponseName':'gamename','Metacritic':'metacritic','IsFree':'isfree','RecommendationCount':'reccount','PCReqsHaveMin':'pcrecs','PriceInitial':'price'})
Game_features_recom = pd.merge(Game_hours_df,Game_feature_df, how='inner',on='gamename')
Game_features_recom= Game_features_recom[['gameid','gamename','metacritic','reccount','isfree','pcrecs','price']]


In [76]:
# LOADING DATA INTO SQL 

# CONNECTION TO MYSQL

rds_connection_string = "postgres:Olajumoke1!@127.0.0.1:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [77]:
engine.table_names()

['games_hours', 'game_recommendation', 'publisher_data']

In [78]:
Game_features_recom.to_sql(name='game_recommendation', con=engine, if_exists='append', index=False)
Game_hours_df.to_sql(name='games_hours', con=engine, if_exists='append', index=False)
Publisher_data.to_sql(name='publisher_data', con=engine, if_exists='append', index=False)

In [79]:
# TEST TO SEE IF CONNECTION IS WORKING

pd.read_sql_query('select * from games_hours', con=engine).head()

,gameid,gamename,gamehours
0,0,Dota 2,4841
1,1,Team Fortress 2,2323
2,2,Counter-Strike Global Offensive,1377
3,3,Unturned,1069
4,4,Left 4 Dead 2,801
